# v7 -> v8

Major differences:

- xmin, xmax, ymin, ymax are integers in Database Units (1nm by default), to set/get them in um You need to use `d` (decimal) Instance.d.xmin which is in float.
- Instance.rotate() takes 0, 1, 2 or 3. For setting arbitrary rotation angles you can use `Instance.d.rotate(30)` to rotate 30deg as an example.
- LayerMap is an Enum of integers
- rewrote routing functions to avoid collisions with objects bounding boxes. `get_route` -> `route_single`. `get_bundle` -> `route_bundle`


Minor details:

- replace `from gdsfactory.cell import cell` with `from gdsfactory import cell`


Improvements:

- gf.routing.route_single can avoid obstacles

## xmin, xmax, ymin, ymax, center in DBU

This is the biggest difference. By default all units are snap to grid.

In [ ]:
import gdsfactory as gf

c = gf.Component()
ref = c << gf.components.bend_euler(radius=5)
c

In [ ]:
print(ref.xmax)
print(ref.d.xmax)

In [ ]:
print(ref.size_info.width)
print(ref.d.size_info.width)

## Use instance.d.rotate for degrees

Instance.rotate() takes 0, 1, 2 or 3. For setting arbitrary rotation angles you can use `Instance.d.rotate(30)` to rotate 30deg as an example.

In [ ]:
c = gf.Component()
ref = c << gf.components.bend_euler(radius=5)
ref.rotate(1)
c

In [ ]:
c = gf.Component()
ref = c << gf.components.bend_euler(radius=5)
ref.d.rotate(45)
c

## LayerMap 

In v7 or below, a LayerMap needs to be called

```python

from gdsfactory.technology import LayerMap

class LayerMapFab(LayerMap):
    WG = (1, 0)

LAYER = LayerMapFab()
```

However in v8 it has a different type and does not need to be called 

```python

from gdsfactory.technology import LayerMap

class LayerMapFab(LayerMap):
    WG = (1, 0)

LAYER = LayerMapFab
```

See below:

In [ ]:
from gdsfactory.technology import LayerMap


class LayerMapFab(LayerMap):
    WG = (1, 0)


LAYER = LayerMapFab
type(LAYER)

In [ ]:
LAYER.WG

In [ ]:
tuple(LAYER.WG)

In [ ]:
str(LAYER.WG)

## Routing functions

Routing functions do no longer return the route Instances but they place the instances in a Component, so you have to pass a Component.

In [ ]:
c = gf.Component()
w = gf.components.straight(cross_section="rib")
top = c << w
bot = c << w
bot.d.move((0, -2))

p0 = top.ports["o2"]
p1 = bot.ports["o2"]

r = gf.routing.route_single(
    c,
    p0,
    p1,
    cross_section="rib",
)
c

In [ ]:
c = gf.Component()
columns = 2
ptop = c << gf.components.pad_array(columns=columns, port_orientation=270)
pbot = c << gf.components.pad_array(port_orientation=270, columns=columns)

ptop.d.movex(300)
ptop.d.movey(300)
routes = gf.routing.route_bundle_electrical(
    c,
    reversed(pbot.ports),
    ptop.ports,
    start_straight_length=100,
    separation=20,
    bboxes=[ptop.bbox(), pbot.bbox()],  # can avoid obstacles
)

c

In [ ]:
c = gf.Component()
columns = 2
ptop = c << gf.components.pad_array(columns=columns, port_orientation=270)
pbot = c << gf.components.pad_array(port_orientation=270, columns=columns)
ptop.d.movex(300)
ptop.d.movey(300)

obstacle = c << gf.c.rectangle(size=(100, 100), layer="M3")
obstacle.d.ymin = pbot.d.ymax
obstacle.d.xmin = pbot.d.xmax + 10


routes = gf.routing.route_bundle_electrical(
    c,
    reversed(pbot.ports),
    ptop.ports,
    start_straight_length=100,
    separation=20,
    bboxes=[obstacle.bbox()],  # can avoid obstacles
)

c